In [1]:
import numpy as np
import pandas as pd
import keras

from keras.layers import LSTM, Input, Dense
from keras.models import Model
from keras.utils import to_categorical

/home/vivekb/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 64        # No of samples in each batch
seq_len = 20           # No of 'block' for each sample in the data. This essentially is the no. of time steps 
inp_size = 7           # No of variables at each time step
hidden_state_dim = 784 # No. of context variables for each sample
lstm_dim = 37          # Equal to hidden_state_dim

### Model Architecture

In [3]:
## Define MLP
mlp_inp = Input(batch_shape=(batch_size, hidden_state_dim))
mlp_dense = Dense(hidden_state_dim, activation='relu')(mlp_inp)
mlp_model = Model(mlp_inp, mlp_dense)

## Define LSTM model
lstm_inp = Input(batch_shape=(batch_size, seq_len, inp_size))
lstm_layer = LSTM(lstm_dim, stateful=True)(lstm_inp)
lstm_out = Dense(10,activation='softmax')(lstm_layer)
lstm_model = Model(lstm_inp, lstm_out)

## Set the output of MLP as initian cell state
lstm_model.layers[1].states[1] = mlp_model.layers[1].output

## Define the final model
model = Model([mlp_inp, lstm_inp], lstm_model.output)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Test the model: Train the LSTM with MNIST 

In [4]:
from keras.datasets import mnist
data = mnist.load_data()

x, y = data[0]
testx, testy = data[1]
print( x.shape,y.shape)

((60000, 28, 28), (60000,))


In [5]:
n_samples = 4096
x_seq = np.random.normal(size=(n_samples, 20, 7))
print(x_seq.shape)

(4096, 20, 7)


### Print weights of MLP after each epoch

In [6]:
for i in range(5):
    print(mlp_model.layers[1].get_weights()[0][1][1])
    model.fit([x[:n_samples].reshape(-1,784), x_seq], 
              to_categorical(y[:n_samples],10), 
              batch_size=batch_size, 
              epochs=1,
              verbose=0)

0.053665955
0.053665955
0.053665955
0.053665955
0.053665955


### Accuracy over epochs

In [7]:
model.fit([x[:n_samples].reshape(-1,784), x_seq], 
              to_categorical(y[:n_samples],10), 
              batch_size=batch_size, 
              epochs=10,
              verbose=1)

Epoch 1/10
4096/4096 [==============================] - 1s - loss: 2.2889 - acc: 0.1326     
Epoch 2/10
4096/4096 [==============================] - 1s - loss: 2.2862 - acc: 0.1335     
Epoch 3/10
4096/4096 [==============================] - 1s - loss: 2.2829 - acc: 0.1372     
Epoch 4/10
4096/4096 [==============================] - 1s - loss: 2.2794 - acc: 0.1411     
Epoch 5/10
4096/4096 [==============================] - 1s - loss: 2.2763 - acc: 0.1426     
Epoch 6/10
4096/4096 [==============================] - 1s - loss: 2.2717 - acc: 0.1489     
Epoch 7/10
4096/4096 [==============================] - 1s - loss: 2.2673 - acc: 0.1516     
Epoch 8/10
4096/4096 [==============================] - 1s - loss: 2.2620 - acc: 0.1531     
Epoch 9/10
4096/4096 [==============================] - 1s - loss: 2.2565 - acc: 0.1570     
Epoch 10/10
4096/4096 [==============================] - 1s - loss: 2.2489 - acc: 0.1687     


### In model summary the MLP part is not included

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (64, 20, 7)               0         
_________________________________________________________________
lstm_1 (LSTM)                (64, 37)                  6660      
_________________________________________________________________
dense_2 (Dense)              (64, 10)                  380       
Total params: 7,040
Trainable params: 7,040
Non-trainable params: 0
_________________________________________________________________
